Agregar features segun el nombre de usuario <br>
Usar la prediccion de un predictor como feature para otro predictor <br>
Usar varios clasificadores y promediar los resultados

In [1]:
import gc
import numpy as np

In [2]:
gc.collect()

0

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from category_encoders import TargetEncoder

In [4]:
import xgboost as xgb

In [5]:
from sklearn.model_selection import GridSearchCV

In [6]:
import pandas as pd

In [7]:
predict = pd.read_csv('trocafone_kaggle_test.csv')

In [8]:
file = pd.read_csv('events_up_to_01062018.csv',low_memory=False)

In [9]:
a_predecir = pd.read_csv('labels_training_set.csv')

In [10]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 23 columns):
timestamp                   object
event                       object
person                      object
url                         object
sku                         float64
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
dtypes: float64(1), object(22)
memory usage: 410.9+ MB


In [11]:
file.person.unique().size

38829

In [12]:
file['timestamp'] = pd.to_datetime(file['timestamp'])

In [13]:
file['event'].value_counts().index[0]

'viewed product'

In [ ]:
def transformacion (x):
    compras = (x['event']=='conversion').sum()
    visitas = (x['event']=='visited site').sum()
    cant_prod_vistos = (x['event']=='viewed product').sum()
    mayor_evento = x['event'].value_counts().index[0]
    dispos = x['device_type'].value_counts().index
    if dispos.size>0:
        dispositivo = dispos[0]
    else:
        dispositivo = 'Ninguno'
    
    prods = x.loc[x.event=='viewed product','model'].value_counts().index
    if prods.size>0:
        prod_visto = prods[0]
    else:
        prod_visto = 'Ninguno'
    
    return pd.Series([compras,visitas,cant_prod_vistos,mayor_evento,dispositivo,prod_visto])
features = file.groupby('person').apply(transformacion)
features.columns = ['compras','visitas','cant_prod_vistos','mayor_evento','dispositivo','prod_visto']

In [ ]:
def temporal (x):
    s = x['timestamp'].size
    primer_visita = x['timestamp'].min()
    ultima_visita = x['timestamp'].max()
    cant_eventos = s
    return pd.Series([primer_visita,ultima_visita,cant_eventos])
temporal = file.groupby('person').apply(temporal)

In [16]:
def channel (x):
    channels = x['channel'].value_counts()
    if channels.size > 0:
        return channels.index[0]
    else:
        return 'Ninguno'
channel = file.groupby('person').apply(channel)
channel.columns = ['channel']

In [17]:
temporal.columns = ['primer_visita','ultima_visita','cant_eventos']
processed = features.join(temporal)

In [18]:
processed.head()

,compras,visitas,cant_prod_vistos,mayor_evento,dispositivo,prod_visto,primer_visita,ultima_visita,cant_eventos
person,,,,,,,,,
0008ed71,0,2,0,checkout,Computer,Ninguno,2018-05-17 12:27:47,2018-05-17 16:28:37,6
00091926,0,34,372,viewed product,Computer,iPhone 6S,2018-05-03 22:08:29,2018-05-31 19:52:03,448
00091a7a,0,1,3,brand listing,Smartphone,iPhone 6,2018-03-26 14:51:11,2018-03-26 14:56:58,10
000ba417,1,6,153,viewed product,Computer,Samsung Galaxy A3 Duos,2018-05-17 11:11:45,2018-05-26 13:09:22,206
000c79fe,0,1,3,searched products,Smartphone,iPhone 7,2018-05-29 00:27:47,2018-05-29 00:38:07,17


In [19]:
processed['permanencia'] = (processed['ultima_visita']-processed['primer_visita']).dt.days + 1

In [20]:
processed['hace_cuanto_visito'] = (file['timestamp'].max()-processed['ultima_visita']).dt.days

In [21]:
processed['eventos_por_dia'] = processed['cant_eventos']/processed['permanencia']

In [22]:
processed['channel'] = channel

In [23]:
processed['visitas_por_dia'] = processed['visitas']/processed['permanencia']

In [24]:
processed.head()

,compras,visitas,cant_prod_vistos,mayor_evento,dispositivo,prod_visto,primer_visita,ultima_visita,cant_eventos,permanencia,hace_cuanto_visito,eventos_por_dia,channel,visitas_por_dia
person,,,,,,,,,,,,,,
0008ed71,0,2,0,checkout,Computer,Ninguno,2018-05-17 12:27:47,2018-05-17 16:28:37,6,1,14,6.0,Referral,2.000000
00091926,0,34,372,viewed product,Computer,iPhone 6S,2018-05-03 22:08:29,2018-05-31 19:52:03,448,28,0,16.0,Direct,1.214286
00091a7a,0,1,3,brand listing,Smartphone,iPhone 6,2018-03-26 14:51:11,2018-03-26 14:56:58,10,1,66,10.0,Paid,1.000000
000ba417,1,6,153,viewed product,Computer,Samsung Galaxy A3 Duos,2018-05-17 11:11:45,2018-05-26 13:09:22,206,10,5,20.6,Direct,0.600000
000c79fe,0,1,3,searched products,Smartphone,iPhone 7,2018-05-29 00:27:47,2018-05-29 00:38:07,17,1,2,17.0,Paid,1.000000


In [25]:
processed = processed.join(a_predecir.set_index('person'))

In [34]:
training_labels = processed['label'].dropna()
training_features = processed.dropna().drop(['primer_visita','ultima_visita','label'],axis=1)

In [35]:
tg = TargetEncoder()
encoded = tg.fit_transform(np.array(training_features),np.array(training_labels))

In [36]:
encoded.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,2,0,0.008565,0.068882,0.042654,6,1,14,6.000000,0.060804,2.000000
1,0,1,3,0.052304,0.035682,0.052632,17,1,2,17.000000,0.042036,1.000000
2,0,1,4,0.026316,0.035682,0.032799,19,1,0,19.000000,0.042036,1.000000
3,2,19,189,0.054229,0.068882,0.025773,471,115,9,4.095652,0.042036,0.165217
4,1,0,2,0.008565,0.017361,0.056452,7,1,9,7.000000,0.017361,0.000000


In [37]:
params={
    'colsample_bylevel':[1,0.5],
    'colsample_bytree':[0.5,1],
    'gamma':[0,5],
    'learning_rate':[0.1,0.3],
    'max_depth':[3,2] 
}
grid = GridSearchCV(xgb.XGBClassifier(),params,scoring='roc_auc',cv=2,verbose=3)

In [38]:
xtrain,xtest,ytrain,ytest = train_test_split(encoded,np.array(training_labels))

In [39]:
grid.fit(xtrain,ytrain)

Fitting 2 folds for each of 32 candidates, totalling 64 fits
[CV] colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_depth=3 
[CV]  colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_depth=3, score=0.7479474864735292, total=   0.8s
[CV] colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_depth=3 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_depth=3, score=0.7431933756093916, total=   0.8s
[CV] colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_depth=2 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.8s remaining:    0.0s


[CV]  colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_depth=2, score=0.7528107508203049, total=   0.5s
[CV] colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_depth=2 
[CV]  colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_depth=2, score=0.7508650559426191, total=   0.6s
[CV] colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.3, max_depth=3 
[CV]  colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.3, max_depth=3, score=0.7170658065138751, total=   0.7s
[CV] colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.3, max_depth=3 
[CV]  colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.3, max_depth=3, score=0.7185908421590054, total=   0.7s
[CV] colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.3, max_depth=2 
[CV]  colsample_bylevel=1, colsample_bytree=0.5, gamma=0, learning_rate=0.3, max_depth=2, score=0.7386731372695519, total=  

[CV]  colsample_bylevel=0.5, colsample_bytree=0.5, gamma=5, learning_rate=0.1, max_depth=3, score=0.7591827069752384, total=   0.6s
[CV] colsample_bylevel=0.5, colsample_bytree=0.5, gamma=5, learning_rate=0.1, max_depth=3 
[CV]  colsample_bylevel=0.5, colsample_bytree=0.5, gamma=5, learning_rate=0.1, max_depth=3, score=0.747347239603763, total=   0.5s
[CV] colsample_bylevel=0.5, colsample_bytree=0.5, gamma=5, learning_rate=0.1, max_depth=2 
[CV]  colsample_bylevel=0.5, colsample_bytree=0.5, gamma=5, learning_rate=0.1, max_depth=2, score=0.752575321740105, total=   0.4s
[CV] colsample_bylevel=0.5, colsample_bytree=0.5, gamma=5, learning_rate=0.1, max_depth=2 
[CV]  colsample_bylevel=0.5, colsample_bytree=0.5, gamma=5, learning_rate=0.1, max_depth=2, score=0.7446232667750031, total=   0.5s
[CV] colsample_bylevel=0.5, colsample_bytree=0.5, gamma=5, learning_rate=0.3, max_depth=3 
[CV]  colsample_bylevel=0.5, colsample_bytree=0.5, gamma=5, learning_rate=0.3, max_depth=3, score=0.7497330506

[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:   46.9s finished


GridSearchCV(cv=2, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'colsample_bylevel': [1, 0.5], 'colsample_bytree': [0.5, 1], 'gamma': [0, 5], 'learning_rate': [0.1, 0.3], 'max_depth': [3, 2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=3)

In [68]:
grid.score(X=xtest,y=ytest)

0.8215494270760694

In [42]:
xgb = grid.best_estimator_

In [44]:
xgb.fit(encoded,np.array(training_labels))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=5, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [45]:
predict_features = processed.loc[processed['label'].isnull()].drop(['primer_visita','ultima_visita','label'],axis=1)

In [47]:
predict_features_encoded = tg.transform(np.array(predict_features))

In [51]:
predicted_labels = xgb.predict(predict_features_encoded)

/home/javier/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [53]:
predict_features['predicted_label'] = predicted_labels

In [63]:
submit = predict.set_index('person')
submit['label'] = predict_features['predicted_label']

In [65]:
submit = submit.reset_index()

In [66]:
submit.to_csv('submit.csv')